In [35]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [17]:
database_name = 'processed_telecom'
table_name= 'xdr_data'

connection_params = { "host": "localhost", "user": "postgres", "password": "00000000",
                    "port": "5432", "database": database_name}

engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")

# str or SQLAlchemy Selectable (select or text object)
sql_query = 'SELECT * FROM xdr_data '

df = pd.read_sql(sql_query, con= engine)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 150001 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150001 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150001 non-null  float64
 5   Dur. (ms)                                 150001 non-null  float64
 6   IMSI                                      150001 non-null  float64
 7   MSISDN/Number                             150001 non-null  float64
 8   IMEI                                      150001 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

Aggregate information per customer


In [41]:
# Convert 'Handset Type' column to string type
df['Handset Type'] = df['Handset Type'].astype(str)

In [42]:
# Aggregate information per customer
aggregated_df = df.groupby('MSISDN/Number').agg({
    
    'TCP DL Retrans. Vol (Bytes)':'mean',
     'Avg RTT DL (ms)': 'mean',
       'Handset Type':lambda x:x.mode()[0],
        'Avg Bearer TP DL (kbps)': 'mean'
       }).reset_index()
print(aggregated_df.head())


   MSISDN/Number  TCP DL Retrans. Vol (Bytes)  Avg RTT DL (ms)  \
0   3.360171e+10                 2.080991e+07       110.000000   
1   3.360171e+10                 5.751940e+05       145.000000   
2   3.360171e+10                 1.798137e+07       104.397853   
3   3.360171e+10                 2.080991e+07        29.000000   
4   3.360172e+10                 2.080991e+07       109.795706   

                        Handset Type  Avg Bearer TP DL (kbps)  
0            Apple iPhone Se (A1723)                23.000000  
1             Apple iPhone 6 (A1586)              7677.000000  
2  Samsung Galaxy S8 Plus (Sm-G955F)              6671.522963  
3        Apple iPhone 7 Plus (A1784)                63.000000  
4       Samsung Galaxy S8 (Sm-G950F)                12.000000  


In [43]:
# Compute top, bottom, and most frequent values
top_tcp_values = df['TCP DL Retrans. Vol (Bytes)'].nlargest(10)
bottom_tcp_values = df['TCP DL Retrans. Vol (Bytes)'].nsmallest(10)
most_frequent_tcp_values = df['TCP DL Retrans. Vol (Bytes)'].value_counts().head(10)
print("Top TCP values:")
print(top_tcp_values)

Top TCP values:
6421      50021024.0
138739    49991791.0
66754     49986834.0
136231    49986038.0
76027     49969605.0
77841     49921268.0
143262    49833677.0
116528    49779123.0
9726      49777538.0
139884    49663906.0
Name: TCP DL Retrans. Vol (Bytes), dtype: float64


In [44]:
top_rtt_values = df['Avg RTT DL (ms)'].nlargest(10)
bottom_rtt_values = df['Avg RTT DL (ms)'].nsmallest(10)
most_frequent_rtt_values = df['Avg RTT DL (ms)'].value_counts().head(10)
print("Top RTT values:")
print(top_rtt_values)

Top RTT values:
475      221.0
3128     221.0
5416     221.0
5824     221.0
8464     221.0
9391     221.0
10456    221.0
14120    221.0
17469    221.0
17842    221.0
Name: Avg RTT DL (ms), dtype: float64


In [45]:
top_throughput_values = df['Avg Bearer TP DL (kbps)'].nlargest(10)
bottom_throughput_values = df['Avg Bearer TP DL (kbps)'].nsmallest(10)
most_frequent_throughput_values = df['Avg Bearer TP DL (kbps)'].value_counts().head(10)
print("Top Throughput values:")
print(top_throughput_values)

Top Throughput values:
18806     49205.0
38826     49205.0
77256     49205.0
89774     49202.0
143234    49202.0
75906     49200.0
120419    49200.0
78071     49198.0
141638    49195.0
87284     49193.0
Name: Avg Bearer TP DL (kbps), dtype: float64


In [46]:
# Compute distribution of average throughput per handset type
throughput_distribution = df.groupby('Handset Type')['Avg Bearer TP DL (kbps)'].mean()
print(throughput_distribution)

Handset Type
A-Link Telecom I. Cubot A5                                                             23184.000000
A-Link Telecom I. Cubot Note Plus                                                       6527.000000
A-Link Telecom I. Cubot Note S                                                          8515.000000
A-Link Telecom I. Cubot Nova                                                           13300.045927
A-Link Telecom I. Cubot Power                                                          13300.045927
                                                                                           ...     
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809       44.500000
Zyxel Communicat. Lte7460                                                              39741.000000
Zyxel Communicat. Sbg3600                                                              13300.045927
Zyxel Communicat. Zyxel Wah7706                                                        

In [47]:
# Compute average TCP retransmission per handset type
tcp_retransmission_per_handset = df.groupby('Handset Type')['TCP DL Retrans. Vol (Bytes)'].mean()
print(tcp_retransmission_per_handset)

Handset Type
A-Link Telecom I. Cubot A5                                                             2.080991e+07
A-Link Telecom I. Cubot Note Plus                                                      6.023490e+05
A-Link Telecom I. Cubot Note S                                                         4.134448e+07
A-Link Telecom I. Cubot Nova                                                           1.358400e+05
A-Link Telecom I. Cubot Power                                                          7.799000e+03
                                                                                           ...     
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809    1.040629e+07
Zyxel Communicat. Lte7460                                                              2.384303e+07
Zyxel Communicat. Sbg3600                                                              2.080991e+07
Zyxel Communicat. Zyxel Wah7706                                                        

In [ ]:
# Check and impute missing values
df[['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)']].fillna(df.mean(), inplace=True)

# Scaling the data
scaler = StandardScaler()
X = scaler.fit_transform(df[['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)']])

# K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Description of clusters
cluster_descriptions = df.groupby('cluster').mean()

# Convert all values to strings
cluster_descriptions = cluster_descriptions.applymap(str)

print(cluster_descriptions)

In [26]:
print(df[['Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)']].dtypes)


Avg RTT DL (ms)                float64
Avg Bearer TP DL (kbps)        float64
TCP DL Retrans. Vol (Bytes)    float64
dtype: object
